### __NLP Disaster Tweets: Fine_Tuned DistilBERT__
> - https://www.kaggle.com/code/hkayesh/twitter-disaster-nlp-with-fine-tuned-distilbert


A pretrained DistilBERT model is finetuned on the Kaggle disaster Tweets dataset.

In [3]:
!pip install transformers
!pip install datasets
!pip install scipy sklearn
!pip install huggingface_hub

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 64.3 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 6.6 MB 52.0 MB/s 
     |████████████████████████████████| 596 kB 75.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


     |████████████████████████████████| 325 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 87.1 MB/s 
     |████████████████████████████████| 136 kB 85.4 MB/s 
     |████████████████████████████████| 1.1 MB 83.1 MB/s 
     |████████████████████████████████| 127 kB 81.4 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 144 kB 75.8 MB/s 
     |████████████████████████████████| 271 kB 84.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

import os
import tensorflow as tf
from datasets import load_dataset, load_from_disk
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from transformers import AutoTokenizer
from transformers import create_optimizer
from transformers import TFAutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers.keras_callbacks import PushToHubCallback
from huggingface_hub import notebook_login

# to be able to see multiple ouputs from sungle cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### __For deploying the trained model to huggingface hub__

In [ ]:
# notebook_login()

In [ ]:
# ! git config --global credential.helper store
# ! apt install git-lfs
# ! git config --global user.email "n.hong@queensu.ca"
# ! git config --global user.name "Nicole-Hong"
# ! git init && git remote add origin && git pull origin main
# ! git remote add origin https://huggingface.co/spaces/starqueen/nlp-disaster-twitter

# for reference: https://huggingface.co/hkayesh/twitter-disaster-nlp

#### __Load Datasets__

In [5]:
training_data_file = "/content/train.csv"
test_data_file = "/content/test.csv"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [6]:
dataset = load_dataset('csv', data_files = [training_data_file])
dataset = dataset['train'].train_test_split(test_size=0.1)
dataset['valid'] = dataset['test']
dataset['test'] = load_dataset('csv', data_files = [test_data_file])['train']

Using custom data configuration default-5898ddb91cbecbf7


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5898ddb91cbecbf7/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-720267b747b570e4


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-720267b747b570e4/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'target'],
        num_rows: 6851
    })
    test: Dataset({
        features: ['id', 'keyword', 'location', 'text'],
        num_rows: 3263
    })
    valid: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'target'],
        num_rows: 762
    })
})

In [8]:
pd.DataFrame(dataset['train'])

,id,keyword,location,text,target
0,1628,bombed,MA,@MisfitRarity misfit got bombed,0
1,6687,landslide,The Circle of Life,So when you're caught in a landslide\nI'll be ...,0
2,6527,injuries,"Georgia, U.S.A.",@msnbc What a fucking idiot. He had a gun &amp...,1
3,6817,loud%20bang,"Bedford, England",The chick I work with chews chewing gum so lou...,0
4,10709,wreck,None,My emotions are a train wreck. My body is a tr...,0
...,...,...,...,...,...
6846,4355,earthquake,oklahoma,Posted a new song: 'Earthquake' http://t.co/Rf...,0
6847,2627,crashed,"Pakistan, Islamabad",Pak Army Helicopter crashed in Mansehra.,1
6848,4905,explode,None,@Anonchimp think its a tie with thunderstorms ...,0
6849,10192,violent%20storm,"ÌÏT: 1.50225,103.742992",Dramatic Video Shows Plane Landing During Viol...,1


In [9]:
pd.DataFrame(dataset['valid'])

,id,keyword,location,text,target
0,7068,meltdown,IL,@crowdtappers @joinvroom OMG I remember the me...,0
1,2238,chemical%20emergency,"Evanston, IL",Emergency units simulate a chemical explosion ...,1
2,1174,blight,UK,The #Palestinian #refugee tragedy is a blight ...,1
3,1418,body%20bag,New York,genuine Leather man Bag Messenger fit iPad min...,0
4,737,attacked,israel,Christian Attacked by Muslims at the Temple Mo...,1
...,...,...,...,...,...
757,2950,danger,Jersey - C.I,Honestly tho Modibo Maiga is stealing a living...,1
758,7773,police,Indonesia,Oops: Bounty hunters try to raid Phoenix polic...,0
759,2883,damage,Somewhere in the Canada,Nine inmates charged with causing damage in Ca...,1
760,8344,ruin,| CA Û¢ GA |,@okgabby_ damn suh. don't let that ruin your y...,0


In [10]:
pd.DataFrame(dataset['test'])

,id,keyword,location,text
0,0,None,None,Just happened a terrible car crash
1,2,None,None,"Heard about #earthquake is different cities, s..."
2,3,None,None,"there is a forest fire at spot pond, geese are..."
3,9,None,None,Apocalypse lighting. #Spokane #wildfires
4,11,None,None,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,None,None,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,None,None,Storm in RI worse than last hurricane. My city...
3260,10868,None,None,Green Line derailment in Chicago http://t.co/U...
3261,10874,None,None,MEG issues Hazardous Weather Outlook (HWO) htt...


#### __Preprocessing Data__

In [11]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

pre_tokenizer_columns = set(dataset["train"].features)
encoded_dataset = dataset.map(tokenize_function, batched=True)
tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Columns added by tokenizer: ['attention_mask', 'input_ids']


In [12]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'target', 'input_ids', 'attention_mask'],
        num_rows: 6851
    })
    test: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'input_ids', 'attention_mask'],
        num_rows: 3263
    })
    valid: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'target', 'input_ids', 'attention_mask'],
        num_rows: 762
    })
})

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = encoded_dataset['train'].to_tf_dataset(
    columns=tokenizer_columns,
    label_cols=["target"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

tf_validation_dataset = encoded_dataset['valid'].to_tf_dataset(
    columns=tokenizer_columns,
    label_cols=["target"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_test_dataset = encoded_dataset['test'].to_tf_dataset(
    columns=tokenizer_columns,
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_test_dataset

<PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>

#### __Fine_Tune the Model__

In [14]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2
)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [15]:
num_epochs = 3
batches_per_epoch = len(encoded_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)
loss = loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [19]:
# start time to measure the time of the program execution
start_time = time.time()

# # Turn on the commented out code below if you want to push the model to huggingface hub

# tensorboard_callback = TensorBoard(log_dir="./text_classification_model_save/logs")
# checkpoint_callback = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

# push_to_hub_model_id = "twitter-disaster-nlp"
# model_output_dir = "./text_classification_model_save"
# push_to_hub_callback = PushToHubCallback(
#     output_dir = model_output_dir,
#     tokenizer = tokenizer,
#     hub_model_id = push_to_hub_model_id,
# )

# callbacks = [push_to_hub_callback]
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=3,
#     callbacks=callbacks,
)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

Epoch 1/3
428/428 [==============================] - 2226s 5s/step - loss: 0.4040 - accuracy: 0.8329 - val_loss: 0.4424 - val_accuracy: 0.7940
Epoch 2/3
428/428 [==============================] - 2239s 5s/step - loss: 0.3087 - accuracy: 0.8795 - val_loss: 0.4694 - val_accuracy: 0.8045
Epoch 3/3
428/428 [==============================] - 2228s 5s/step - loss: 0.2382 - accuracy: 0.9159 - val_loss: 0.4788 - val_accuracy: 0.8058




--- 6693.032206773758 seconds ---


#### __Make Prediction__

In [21]:
# start time to measure the time of the program execution
start_time = time.time()

test_pred = model.predict(tf_test_dataset)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")



--- 334.58309841156006 seconds ---


In [22]:
lsubmission = pd.read_csv('/content/sample_submission.csv')
submission['target'] = np.argmax(test_pred.logits, axis=1)
submission.to_csv('trial15_submission_pre-trained_distilbert.csv', index=False)